In [25]:
import numpy as np
import pandas as pd
import importlib

import ipywidgets as widgets

import pysindy as ps
import do_mpc.model as mpc_model
import do_mpc.controller as mpc_contr

In [3]:
import cpagent
import cprender
import cpenvs
import cppid

_ = importlib.reload(cppid)
_ = importlib.reload(cpagent)
_ = importlib.reload(cprender)
_ = importlib.reload(cpenvs)

In [4]:
# Baseline agent
agent = cppid.PidAgentMoving2(
    (0.301, 0.0190, 0.0797),
    (0.0659, 0.00, 0.0499),
    1.33
)
df = cpagent.execute_cartpole(agent, env=cpenvs.MovingCartpoleEnv(), num_episodes=500)
df

,ep,t,cart_pos,cart_vel,pole_ang,pole_vel,pos_deviation,force,reward,cart_pos_setpoint
0,0,0,-0.036138,0.007657,0.002915,-0.031009,-0.036138,0.0,NaN,0.000000
1,0,1,-0.035985,-0.187507,0.002294,0.262592,-0.035985,-10.0,0.999353,0.000000
2,0,2,-0.039735,-0.382662,0.007546,0.555998,-0.039735,-10.0,0.999211,0.000000
3,0,3,-0.047388,-0.187646,0.018666,0.265702,-0.047388,10.0,0.998877,0.000000
4,0,4,-0.051141,0.007204,0.023980,-0.021035,-0.051141,10.0,0.998692,0.000000
...,...,...,...,...,...,...,...,...,...,...
250495,499,496,-0.929140,0.369910,0.011134,-0.543009,-0.019890,10.0,0.999802,-0.909249
250496,499,497,-0.921741,0.174633,0.000274,-0.246838,-0.012492,-10.0,0.999922,-0.909249
250497,499,498,-0.918249,-0.020493,-0.004663,0.045931,-0.009000,-10.0,0.999960,-0.909249
250498,499,499,-0.918659,-0.215548,-0.003744,0.337139,-0.009409,-10.0,0.999956,-0.909249


In [5]:
trajectories_x = []
trajectories_xdot = []
trajectories_u = []


for ep in range(100):
    dff = df.loc[df['ep']==ep]
    trajectories_x.append(dff[["cart_pos", "cart_vel", "pole_ang", "pole_vel"]].to_numpy())
    #trajectories_x.append(dff[["cart_pos", "pole_ang"]].to_numpy())
    #trajectories_xdot.append(dff[["cart_vel", "pole_vel"]].to_numpy())
    trajectories_u.append(dff[["force"]].to_numpy())

In [67]:
sindy_functions = [
    lambda x : x,
    lambda x : x * x,
    lambda x, y : x*y,
    lambda x, y : (x*x)*y,
    lambda x, y : x* (y*y),
]

sindy_function_names = [
    lambda x_name : f"{x_name}",
    lambda x_name : f"{x_name}^2",
    lambda x_name, y_name : f"{x_name} * {y_name}",
    lambda x_name, y_name : f"{x_name}^2 * {y_name}",
    lambda x_name, y_name : f"{x_name} * {y_name}^2",

]

sindy_featurelib = ps.feature_library.CustomLibrary(sindy_functions, sindy_function_names)

mySindy = ps.SINDy(
    #feature_names=["cart_pos",  "pole_ang", "force"],
    feature_names=["cart_pos", "cart_vel", "pole_ang", "pole_vel", "force"],
    t_default = 0.02,
    feature_library=sindy_featurelib
)

mySindy.fit(
    multiple_trajectories=True,
    x=trajectories_x, 
    #x_dot=trajectories_xdot,
    u=trajectories_u,
)

c:\Source\studienarbeit-ml-controls\.env\lib\site-packages\pysindy\feature_library\base.py:40: DeprecationWarning:

Library ensembling is no longer performed by feature libraries.  Use EnsemblingOptimizer to fit an ensemble model.

c:\Source\studienarbeit-ml-controls\.env\lib\site-packages\pysindy\differentiation\finite_difference.py:199: DeprecationWarning:

`np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`



SINDy(differentiation_method=FiniteDifference(),
      feature_library=<pysindy.feature_library.custom_library.CustomLibrary object at 0x000001590BD2DED0>,
      feature_names=['cart_pos', 'cart_vel', 'pole_ang', 'pole_vel', 'force'],
      optimizer=STLSQ(), t_default=0.02)

In [68]:
mySindy.print()

(cart_pos)' = 1.042 cart_vel + -0.412 pole_ang + 0.139 pole_vel
(cart_vel)' = 159.386 cart_pos + 3.246 cart_vel + 228.443 pole_ang + -54.201 pole_vel + 0.896 force + -0.325 cart_vel^2 + -4.027 pole_ang^2 + -0.145 pole_vel^2 + 0.304 cart_pos * cart_vel + -3.080 cart_pos * pole_ang + 2.929 cart_vel * pole_ang + -0.490 cart_vel * pole_vel + 3.326 pole_ang * pole_vel + 6.130 cart_pos^2 * cart_vel + 54.257 cart_pos^2 * pole_ang + 7.203 cart_pos^2 * pole_vel + 30.089 cart_vel^2 * pole_ang + 6.709 cart_vel^2 * pole_vel + -38.281 pole_ang^2 * pole_vel + 0.282 pole_ang^2 * force + -0.680 pole_vel^2 * force + 4.596 cart_pos * cart_vel^2 + 168.318 cart_pos * pole_ang^2 + -2.595 cart_pos * pole_vel^2 + -1.594 cart_pos * force^2 + -99.551 cart_vel * pole_ang^2 + 9.950 cart_vel * pole_vel^2 + -30.853 pole_ang * pole_vel^2 + -2.253 pole_ang * force^2 + 0.648 pole_vel * force^2
(pole_ang)' = 0.485 pole_ang + 0.846 pole_vel
(pole_vel)' = -238.960 cart_pos + -5.507 cart_vel + -327.742 pole_ang + 81.175 

In [69]:
print(len(mySindy.get_feature_names()))
mySindy.get_feature_names()

40


['cart_pos',
 'cart_vel',
 'pole_ang',
 'pole_vel',
 'force',
 'cart_pos^2',
 'cart_vel^2',
 'pole_ang^2',
 'pole_vel^2',
 'force^2',
 'cart_pos * cart_vel',
 'cart_pos * pole_ang',
 'cart_pos * pole_vel',
 'cart_pos * force',
 'cart_vel * pole_ang',
 'cart_vel * pole_vel',
 'cart_vel * force',
 'pole_ang * pole_vel',
 'pole_ang * force',
 'pole_vel * force',
 'cart_pos^2 * cart_vel',
 'cart_pos^2 * pole_ang',
 'cart_pos^2 * pole_vel',
 'cart_pos^2 * force',
 'cart_vel^2 * pole_ang',
 'cart_vel^2 * pole_vel',
 'cart_vel^2 * force',
 'pole_ang^2 * pole_vel',
 'pole_ang^2 * force',
 'pole_vel^2 * force',
 'cart_pos * cart_vel^2',
 'cart_pos * pole_ang^2',
 'cart_pos * pole_vel^2',
 'cart_pos * force^2',
 'cart_vel * pole_ang^2',
 'cart_vel * pole_vel^2',
 'cart_vel * force^2',
 'pole_ang * pole_vel^2',
 'pole_ang * force^2',
 'pole_vel * force^2']

In [70]:
mySindy.coefficients()

array([[ 0.00000000e+00,  1.04151285e+00, -4.12280088e-01,
         1.39348428e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.59385924e+02,  3.24638927e+00,  2.28442720e+02,
        -5.42009237e+01,  8.95566533e-01,  0.00000000e+00,
        -3.25402007e-01, -4.02661614e+00, -1.44638226e-01,
         0.00000000e+00,  3.03

In [71]:
mySindy.coefficients()[1, :]

array([ 1.59385924e+02,  3.24638927e+00,  2.28442720e+02, -5.42009237e+01,
        8.95566533e-01,  0.00000000e+00, -3.25402007e-01, -4.02661614e+00,
       -1.44638226e-01,  0.00000000e+00,  3.03946646e-01, -3.08041627e+00,
        0.00000000e+00,  0.00000000e+00,  2.92863936e+00, -4.90301096e-01,
        0.00000000e+00,  3.32618442e+00,  0.00000000e+00,  0.00000000e+00,
        6.13003136e+00,  5.42572473e+01,  7.20320262e+00,  0.00000000e+00,
        3.00888700e+01,  6.70878307e+00,  0.00000000e+00, -3.82814581e+01,
        2.81534701e-01, -6.79520364e-01,  4.59602885e+00,  1.68317652e+02,
       -2.59459243e+00, -1.59353242e+00, -9.95507584e+01,  9.95022372e+00,
        0.00000000e+00, -3.08533924e+01, -2.25261481e+00,  6.48221591e-01])

In [72]:
myModel = mpc_model.Model(model_type="continuous")
cart_pos = myModel.set_variable(var_type='_x', var_name='cart_pos', shape=(1,1))
cart_vel = myModel.set_variable(var_type='_x', var_name='cart_vel', shape=(1,1))
pole_ang = myModel.set_variable(var_type='_x', var_name='pole_ang', shape=(1,1))
pole_vel = myModel.set_variable(var_type='_x', var_name='pole_vel', shape=(1,1))
u__force = myModel.set_variable(var_type='_u', var_name='u__force', shape=(1,1))

modelvars_x = [cart_pos, cart_vel, pole_ang, pole_vel]
modelvars_u = [u__force]
modelvars = modelvars_x + modelvars_u

In [73]:
# Output should match output of "mySindy.get_feature_names()"

from itertools import combinations
sindy_candidates = []

for i, f in enumerate(sindy_functions):
    for c in combinations(range(len(sindy_functions)), f.__code__.co_argcount):
        v = [modelvars[i] for i in c]
        sindy_candidates.append(f(*v))

# Print
print(len(sindy_candidates))
for c in sindy_candidates:
    print(c)

40
cart_pos
cart_vel
pole_ang
pole_vel
u__force
sq(cart_pos)
sq(cart_vel)
sq(pole_ang)
sq(pole_vel)
sq(u__force)
(cart_pos*cart_vel)
(cart_pos*pole_ang)
(cart_pos*pole_vel)
(cart_pos*u__force)
(cart_vel*pole_ang)
(cart_vel*pole_vel)
(cart_vel*u__force)
(pole_ang*pole_vel)
(pole_ang*u__force)
(pole_vel*u__force)
(sq(cart_pos)*cart_vel)
(sq(cart_pos)*pole_ang)
(sq(cart_pos)*pole_vel)
(sq(cart_pos)*u__force)
(sq(cart_vel)*pole_ang)
(sq(cart_vel)*pole_vel)
(sq(cart_vel)*u__force)
(sq(pole_ang)*pole_vel)
(sq(pole_ang)*u__force)
(sq(pole_vel)*u__force)
(cart_pos*sq(cart_vel))
(cart_pos*sq(pole_ang))
(cart_pos*sq(pole_vel))
(cart_pos*sq(u__force))
(cart_vel*sq(pole_ang))
(cart_vel*sq(pole_vel))
(cart_vel*sq(u__force))
(pole_ang*sq(pole_vel))
(pole_ang*sq(u__force))
(pole_vel*sq(u__force))


In [74]:
for i, v in enumerate(modelvars_x):
    rhs = 0.0
    coeffs_vector = mySindy.coefficients()[i, :]
    for j, cand in enumerate(sindy_candidates):
        coeff = coeffs_vector[j]
        if abs(coeff) >= 0.0001:
            rhs = rhs + coeff * cand
    print(f"({v.name()})' = {rhs}")
    myModel.set_rhs(v.name(), rhs)

myModel.setup()

(cart_pos)' = (((1.04151*cart_vel)+(-0.41228*pole_ang))+(0.139348*pole_vel))
(cart_vel)' = ((((((((((((((((((((((((((((((159.386*cart_pos)+(3.24639*cart_vel))+(228.443*pole_ang))+(-54.2009*pole_vel))+(0.895567*u__force))+(-0.325402*sq(cart_vel)))+(-4.02662*sq(pole_ang)))+(-0.144638*sq(pole_vel)))+(0.303947*(cart_pos*cart_vel)))+(-3.08042*(cart_pos*pole_ang)))+(2.92864*(cart_vel*pole_ang)))+(-0.490301*(cart_vel*pole_vel)))+(3.32618*(pole_ang*pole_vel)))+(6.13003*(sq(cart_pos)*cart_vel)))+(54.2572*(sq(cart_pos)*pole_ang)))+(7.2032*(sq(cart_pos)*pole_vel)))+(30.0889*(sq(cart_vel)*pole_ang)))+(6.70878*(sq(cart_vel)*pole_vel)))+(-38.2815*(sq(pole_ang)*pole_vel)))+(0.281535*(sq(pole_ang)*u__force)))+(-0.67952*(sq(pole_vel)*u__force)))+(4.59603*(cart_pos*sq(cart_vel))))+(168.318*(cart_pos*sq(pole_ang))))+(-2.59459*(cart_pos*sq(pole_vel))))+(-1.59353*(cart_pos*sq(u__force))))+(-99.5508*(cart_vel*sq(pole_ang))))+(9.95022*(cart_vel*sq(pole_vel))))+(-30.8534*(pole_ang*sq(pole_vel))))+(-2.25261*(p

In [75]:
mpc = mpc_contr.MPC(myModel)

setup_mpc = {
    'n_horizon': 20,
    't_step': 0.1,
    'n_robust': 1,
    'store_full_solution': True,
}
mpc.set_param(**setup_mpc)

# mterm = cart_pos**2 + pole_ang**2 
# lterm = cart_pos**2 + pole_ang**2 
mterm = pole_ang**2 
lterm = cart_pos**2 + pole_ang**2 
mpc.set_objective(mterm=mterm, lterm=lterm)

mpc.bounds['lower','_u', 'u__force'] = -10.0
mpc.bounds['upper','_u', 'u__force'] = 10.0
mpc.setup()

In [76]:
class MPCCartPoleAgentCont(cpagent.CartPoleAgentABC):
    def __init__(self, mpc: mpc_contr.MPC) -> None:
        self.mpc = mpc

    def step(self, env_state: np.ndarray) -> int|float:
        u = mpc.make_step(np.array([
            1.0*env_state[0],
            env_state[1],
            env_state[2],
            env_state[3],
        ]))
        return u[0]

In [77]:
# Baseline agent
mpcagent = MPCCartPoleAgentCont(mpc)
dfmpc = cpagent.execute_cartpole(agent, env=cpenvs.MovingCartpoleEnvEnvCont(), num_episodes=10)
dfmpc

,ep,t,cart_pos,cart_vel,pole_ang,pole_vel,pos_deviation,force,reward,cart_pos_setpoint
0,0,0,0.014450,-0.020729,0.000600,0.025387,0.014450,0.0,NaN,0.0
1,0,1,0.014036,0.174384,0.001108,-0.267107,0.014036,10.0,0.999901,0.0
2,0,2,0.017524,0.369490,-0.004234,-0.559440,0.017524,10.0,0.999846,0.0
3,0,3,0.024913,0.369550,-0.015423,-0.560775,0.024913,-10.0,0.999690,0.0
4,0,4,0.032304,0.369766,-0.026639,-0.565633,0.032304,-10.0,0.999478,0.0
...,...,...,...,...,...,...,...,...,...,...
375,9,27,0.154876,0.396708,-0.386719,-1.597940,0.154876,-10.0,0.988007,0.0
376,9,28,0.162810,0.400736,-0.418678,-1.714419,0.162810,-10.0,0.986746,0.0
377,9,29,0.170825,0.404848,-0.452967,-1.839580,0.170825,-10.0,0.985409,0.0
378,9,30,0.178922,0.408988,-0.489758,-1.973829,0.178922,-10.0,0.983993,0.0


In [78]:
EP = 0

fig = cprender.lineplot(dfmpc, ep=EP, incl_velo=False)
fig.show()